In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
from torch import nn
import timm
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from torch.utils.data import Dataset,DataLoader
import h5py
from io import BytesIO
from PIL import Image
from torchvision.transforms import v2
import os
from sklearn.model_selection import GroupShuffleSplit , train_test_split, StratifiedGroupKFold
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score,roc_curve,auc
import gc
import random
from torchvision.ops import sigmoid_focal_loss
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import pickle
from sklearn.preprocessing import StandardScaler

In [10]:
print('\nDensenet models: ', timm.list_models('*efficientnet_b3*'))


Densenet models:  ['efficientnet_b3', 'efficientnet_b3_g8_gn', 'efficientnet_b3_gn', 'efficientnet_b3_pruned', 'tf_efficientnet_b3']


In [11]:
MODEL_NAME = 'efficientnet_b3'
OVER_SAMPLE_RATIO = 0.01
UNDER_SAMPLE_RATIO = 1
CNN_EMBEDDINGS_SIZE = 1
VAL_AFTER = 5
DROPOUT_RATE = 0.6
TABULAR_SHARE = 512
IMAGE_INPUT_SHAPE = 224
BATCH_SIZE = 32
VAL_BATCH_SIZE = 64
EPOCHS = 15
SPLITS = 5
BEST_WEIGHT = '/kaggle/input/lets-go-neural-8-image-only-state-dict/fullmodel2_epoch1loss0.130auc0.881pauc0.158.pth'
SEED = 42
ALPHA = 0.01
GAMMA = 2
FULL = 0.025
# SAVE_PATH = '/kaggle/working/fullmodel/'

In [12]:

cnn_model = timm.create_model(model_name=MODEL_NAME,pretrained=False,num_classes=CNN_EMBEDDINGS_SIZE)
cnn_model


EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNormAct2d(
          40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (b

In [13]:
# with torch.inference_mode():
#     cnn_model.eval()
#     print(cnn_model(torch.rand(1,3,384,384)).shape)

In [14]:
train_meta = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv',low_memory=False)
test_meta = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv',low_memory=False)

In [15]:
test_cols = test_meta.columns
test_cols

Index(['isic_id', 'patient_id', 'age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'image_type', 'tbp_tile_type', 'tbp_lv_A',
       'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext',
       'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA',
       'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location',
       'tbp_lv_location_simple', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'attribution', 'copyright_license'],
      dtype='object')

In [16]:
[(len(set(train_meta[col])),col) for col in test_cols if test_meta[col].dtype==object]

[(401059, 'isic_id'),
 (1042, 'patient_id'),
 (3, 'sex'),
 (6, 'anatom_site_general'),
 (1, 'image_type'),
 (2, 'tbp_tile_type'),
 (21, 'tbp_lv_location'),
 (8, 'tbp_lv_location_simple'),
 (7, 'attribution'),
 (3, 'copyright_license')]

In [17]:
cat_cols = [col for col in test_cols if col not in ['isic_id','patient_id','copyright_license','attribution','image_type'] and test_meta[col].dtype=='object']

In [18]:
num_cols = [col for col in test_cols if test_meta[col].dtype!=object]

In [19]:
train_img_path = '/kaggle/input/isic-2024-challenge/train-image.hdf5'
test_img_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
val_img_path = '/kaggle/input/isic-2024-challenge/train-image.hdf5'

In [20]:
def preprocess(train,test,num_cols,cat_cols):
    
    new_train = train.copy()
    new_test = test.copy()
    
#     important columns to return
    group_col = 'patient_id'
    target_col = 'target'
    
#     keeping workable columns
    new_train=new_train[num_cols+cat_cols+['isic_id',group_col,target_col]]
    new_test=new_test[num_cols+cat_cols+['isic_id',group_col]]
    
#     missing values
    imputer_cat = SimpleImputer(strategy='most_frequent')
    imputer_num = SimpleImputer(strategy='median')
    new_train[cat_cols]=imputer_cat.fit_transform(new_train[cat_cols])
    new_train[num_cols]=imputer_num.fit_transform(new_train[num_cols])
    new_test[cat_cols]=imputer_cat.transform(new_test[cat_cols])
    new_test[num_cols]=imputer_num.transform(new_test[num_cols])

#     cat_cols encoding
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=np.nan)
    new_train[cat_cols] = encoder.fit_transform(new_train[cat_cols])
    new_test[cat_cols] = encoder.transform(new_test[cat_cols])
    new_test[cat_cols]=imputer_cat.transform(new_test[cat_cols])

#     removing num_cols with 0 std
    zero_std_cols = [col for col in num_cols if new_train[col].std()==0]
    new_train.drop(columns=zero_std_cols,inplace=True)
    new_test.drop(columns=zero_std_cols,inplace=True)
    num_cols = [col for col in num_cols if col not in zero_std_cols]
    f_cols = cat_cols+num_cols
    
#     get cat_sizes for embeddings
    cat_sizes = [len(set(new_train[col])) for col in cat_cols]
    for col in cat_cols:
        print(set(new_test[col])-set(new_train[col]))
    
#     normalizing train and test
#     means = pd.Series([new_train[col].mean() for col in num_cols],index=num_cols)
#     stds = pd.Series([new_train[col].std() for col in num_cols],index=num_cols)
#     new_train[num_cols] =( new_train[num_cols]-means)/stds
#     new_test[num_cols] =( new_test[num_cols]-means)/stds

#     reset index
    new_train = new_train.reset_index(drop=True)
    
    
    return new_train,new_test,f_cols,group_col,target_col,cat_sizes

In [21]:
tabular_train,tabular_test,f_cols,group_col,target_col,cat_sizes= preprocess(train_meta,test_meta,num_cols,cat_cols)

set()
set()
set()
set()
set()


In [22]:
tabular_train_copy = tabular_train.copy()

In [23]:
cat_sizes

[2, 5, 2, 21, 8]

In [24]:
tabular_test.isna().sum().sum()

0

In [25]:
class FullDataset(Dataset):
    def __init__(self,y,img_paths,transforms = None):
        self.y = y
        self.img_paths = h5py.File(img_paths,mode='r')
        self.transforms = transforms
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        isic_id = self.y.loc[idx,'isic_id']
        image = np.array(Image.open(BytesIO(self.img_paths[isic_id][()])))
        # Extract the V channel from the HSV color space
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)  # Second channel: V channel from HSV
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        to_return = {'img':image}
        if target_col in self.y.columns :
            target = self.y.loc[idx,target_col]
            to_return['target']=torch.tensor(target,dtype=torch.float)
        return to_return

In [42]:
class FullModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.img = timm.create_model(model_name=MODEL_NAME,pretrained=False,num_classes=CNN_EMBEDDINGS_SIZE)
    def forward(self,x):
#         print(x.shape)
#         print_gpu_usage()
        return self.img(x)
FullModel()

FullModel(
  (img): EfficientNet(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNormAct2d(
            40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d

In [27]:
train_transform = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2,contrast_limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),

    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(IMAGE_INPUT_SHAPE, IMAGE_INPUT_SHAPE),
    # A.Cut(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
    A.Normalize(),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(IMAGE_INPUT_SHAPE, IMAGE_INPUT_SHAPE),
    A.Normalize(),
    ToTensorV2(),
])

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/blur/transforms.py:180: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [28]:
# torch.cuda.manual_seed(SEED)
# torch.manual_seed(SEED)
# random.seed(SEED)
# print(f'train_size before: {tabular_train_copy.shape[0]}')
# sgkf = StratifiedGroupKFold(n_splits=5)
# # split = splitter.split(tabular_train, groups=tabular_train[group_col])
# for i, (train_index, test_index) in enumerate(sgkf.split(tabular_train_copy, tabular_train_copy[target_col], tabular_train_copy[group_col])):
#     tabular_val = tabular_train_copy.iloc[test_index].reset_index(drop=True)
#     tabular_train = tabular_train_copy.iloc[train_index].reset_index(drop=True)
#     break
# print(f'train_size after: {tabular_train.shape[0]}')
# n_neg,n_pos = (tabular_train[target_col]==0).sum(),(tabular_train[target_col]==1).sum()
# print(f'n_neg: {n_neg}, n_pos: {n_pos} in train')
# n_neg,n_pos = (tabular_val[target_col]==0).sum(),(tabular_val[target_col]==1).sum()
# print(f'n_neg: {n_neg}, n_pos: {n_pos} in val')
# if FULL!=1:
# #     class_counts = tabular_train_copy[tabular_train_copy[target_col]==0][group_col].value_counts()

# #     # Filter out classes with only 1 member
# #     filtered_df = tabular_train_copy[tabular_train_copy[target_col]==0][tabular_train_copy[tabular_train_copy[target_col]==0][group_col].isin(class_counts[class_counts > 1].index)].reset_index(drop=True)

# #     X_neg,_ = train_test_split(filtered_df,train_size=FULL,shuffle=True,random_state=SEED,stratify=filtered_df[group_col])
# #     tabular_train = pd.concat([X_neg,tabular_train_copy[tabular_train_copy[target_col]==1]],axis=0)
# #     tabular_train=tabular_train.reset_index(drop=True)
#     sampler = RandomUnderSampler()
#     tabular_train,_ = sampler.fit_resample(tabular_train,tabular_train[target_col])
#     tabular_train = tabular_train.reset_index(drop=True)
# #     print(f'train_size middle: {tabular_train.shape[0]}')
# #     faiss = pd.read_csv('/kaggle/input/faiss400/modified_dataset_6.csv')
# #     tabular_train = tabular_train[tabular_train['isic_id'].isin(faiss['isic_id'])].reset_index(drop=True)
# print(f'train_size even after: {tabular_train.shape[0]}')
# n_neg,n_pos = (tabular_train[target_col]==0).sum(),(tabular_train[target_col]==1).sum()
# print(f'n_neg: {n_neg}, n_pos: {n_pos} in train')
# n_neg,n_pos = (tabular_val[target_col]==0).sum(),(tabular_val[target_col]==1).sum()
# print(f'n_neg: {n_neg}, n_pos: {n_pos} in val')

In [29]:
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
# train_dataset = FullDataset(tabular_train[['isic_id',target_col]],train_img_path,transforms = train_transform)
test_dataset = FullDataset(tabular_test[['isic_id']],test_img_path,transforms = test_transform)
# val_dataset = FullDataset(tabular_val[['isic_id',target_col]],val_img_path,transforms = test_transform)

In [30]:
# torch.cuda.manual_seed(SEED)
# torch.manual_seed(SEED)
# random.seed(SEED)
# plt.imshow(np.array(train_dataset[300]['img'].permute(1,2,0)))

In [31]:
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
# train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE,shuffle=True,num_workers=os.cpu_count())
test_loader = DataLoader(test_dataset,batch_size = BATCH_SIZE,shuffle=False,num_workers=os.cpu_count())
# val_loader = DataLoader(val_dataset,batch_size = BATCH_SIZE,shuffle=False,num_workers=os.cpu_count())

In [32]:
def pauc_metric(y_true, y_scores, tpr_threshold=0.8):

    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)

    # Create a mask for TPR values above the threshold
    mask = tpr >= tpr_threshold

    # Filter FPR and TPR values based on the mask
    fpr_above_threshold = fpr[mask]
    tpr_above_threshold = tpr[mask]

    # Calculate the partial AUC
    try:
        partial_auc = auc(fpr_above_threshold, tpr_above_threshold)
    except:
        return 0

    # Normalize the partial AUC
    pauc = partial_auc * (1 - tpr_threshold)

    return pauc 

In [33]:
# def train_epoch(model,train_loader,DEVICE):
#     model.train()
#     ys=[]
#     preds=[]
#     torch.cuda.empty_cache()
#     for i,data in tqdm(enumerate(train_loader),total=len(train_loader)):
#         y = data['target']
#         ys.append(y)
#         y=y.to(DEVICE)
#         img = data['img'].to(DEVICE)
#         y_pred = full_model(img)
#         preds.append(y_pred.squeeze().cpu())
#         loss = loss_fn(y_pred.squeeze(),y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         del img,data
# #     scheduler.step()
#     torch.cuda.empty_cache()
#     gc.collect()
#     model.eval()
#     with torch.inference_mode():
#         ys = torch.cat(ys)
#         preds = torch.cat(preds)
#         pred_probs = nn.Sigmoid()(preds)
#         loss = loss_fn(preds,ys)
#         auc = roc_auc_score(ys,pred_probs)
#         pauc = pauc_metric(ys,pred_probs)
#     return loss,auc,pauc

In [34]:
# def val_epoch(model,val_loader,DEVICE):
#     model.eval()
#     with torch.inference_mode():
#         ys=[]
#         preds=[]
#         torch.cuda.empty_cache()
#         for i,data in tqdm(enumerate(val_loader),total=len(val_loader)):
#             y = data['target']
#             ys.append(y)
#             y=y.to(DEVICE)
#             img = data['img'].to(DEVICE)
#             y_pred = full_model(img)
#             preds.append(y_pred.squeeze().cpu())
#             del img,data
#         torch.cuda.empty_cache()
#         gc.collect()
#         ys = torch.cat(ys)
#         preds = torch.cat(preds)
#         pred_probs = nn.Sigmoid()(preds)
#         loss = loss_fn(preds,ys)
#         auc = roc_auc_score(ys,pred_probs)
#         pauc = pauc_metric(ys,pred_probs)
        
#     return loss,auc,pauc

In [35]:
def predict(model,test_loader,DEVICE):
    model.eval()
    with torch.inference_mode():
        preds=[]
        torch.cuda.empty_cache()
        for i,data in tqdm(enumerate(test_loader),total=len(test_loader)):
            img = data['img'].to(DEVICE)
            y_pred = model(img)
            preds.append(y_pred.squeeze().detach().cpu())
            del img,y_pred,data
            if i%50==0: gc.collect()
        torch.cuda.empty_cache()
        gc.collect()
        preds = torch.cat(preds)
        pred_probs = nn.Sigmoid()(preds)
        
    return pred_probs

In [36]:
# def train_model(model,train_loader,val_loader,DEVICE,epochs=5):
#     best_auc = -2
#     best_pauc = -2
#     train_loss,val_loss=[],[]
#     train_auc,val_auc=[],[]
#     train_pauc,val_pauc=[],[]
#     for e in (range(epochs)):
#         print(f'## Epoch: {e} ongoing ---------------')
#         t_loss,t_auc,t_pauc = train_epoch(model,train_loader,DEVICE)
#         v_loss,v_auc,v_pauc = val_epoch(model,val_loader,DEVICE)
#         train_loss.append(t_loss)
#         val_loss.append(v_loss)
#         train_auc.append(t_auc)
#         val_auc.append(v_auc)
#         train_pauc.append(t_pauc)
#         val_pauc.append(v_pauc)
#         print(f'Epoch: {e}, Train loss: {t_loss}, Train auc: {t_auc}, Train pauc: {t_pauc} | Val loss: {v_loss}, Val auc: {v_auc}, Val pauc: {v_pauc}')
#         if v_pauc>=best_pauc or v_auc>=best_auc:
#             best_pauc,best_auc = max(best_pauc,v_pauc),max(best_auc,v_auc)
#             print('Saving model')
#             torch.save({
#                 'model_state_dict': full_model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#             },f'fullmodel2_epoch{e}loss{v_loss:.3f}auc{v_auc:.3f}pauc{v_pauc:.3f}.pth')
#     return train_loss,val_loss,train_auc,val_auc,train_pauc,val_pauc,model

In [37]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [38]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss

In [52]:
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# full_model = FullModel()
# full_model = full_model.to(DEVICE)
# checkpoint = torch.load(BEST_WEIGHT)
# full_model.load_state_dict(checkpoint['model_state_dict'])
# loss_fn = FocalLoss()
loss_fn = nn.BCEWithLogitsLoss()
pred_probs=np.zeros(len(test_dataset))
scaler = StandardScaler()
for i in range(5):
    print('epoch',i)
    model = FullModel()
    model.to(DEVICE)
    model.load_state_dict(torch.load(f"/kaggle/input/final-train-preds-149/fullmodel2_split{i}.pth")['model_state_dict'])
    preds_i = np.array(predict(model,test_loader,DEVICE)).squeeze()
#     scaled_preds = scaler.fit_transform(preds_i.reshape(-1,1)).squeeze()
    pred_probs+=preds_i
pred_probs/=5

epoch 0


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 2


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 3


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 4


  0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
# v_loss,v_auc,v_pauc = val_epoch(full_model,val_loader,DEVICE)
# print("v_loss,v_auc,v_pauc : ",v_loss,v_auc,v_pauc)

In [54]:
# pred_probs = predict(full_model,test_loader,DEVICE)
# sample_sub = pd.read_csv('/kaggle/input/isic-2024-challenge/sample_submission.csv')
tabular_test['target']=pred_probs
tabular_test[['isic_id','target']].to_csv('submission.csv',index=False)

In [55]:
!head submission.csv

isic_id,target
ISIC_0015657,0.003360839397646487
ISIC_0015729,0.0006716647854773328
ISIC_0015740,0.008018181280931458


In [ ]:
# torch.cuda.manual_seed(SEED)
# torch.manual_seed(0)
# random.seed(0)
# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# full_model = FullModel(zip(cat_sizes,cat_sizes),len(num_cols))
# full_model = full_model.to(DEVICE)
# optimizer = torch.optim.AdamW(full_model.parameters())
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000, eta_min=0, last_epoch=-1, verbose=False)
# loss_fn = FocalLoss()
# train_loss,val_loss,train_auc,val_auc,train_pauc,val_pauc,trained_model = train_model(full_model,train_loader,val_loader,DEVICE,epochs=EPOCHS)

In [ ]:
# plt.plot(range(EPOCHS),train_loss,label ='Train')
# plt.plot(range(EPOCHS),val_loss,label='Val')
# plt.legend()
# plt.title('Loss')

In [ ]:
# plt.plot(range(EPOCHS),train_auc,label ='Train')
# plt.plot(range(EPOCHS),val_auc,label='Val')
# plt.legend()
# plt.title('Roc_Auc')

In [ ]:
# plt.plot(range(EPOCHS),train_pauc,label ='Train')
# plt.plot(range(EPOCHS),val_pauc,label='Val')
# plt.legend()
# plt.title('Roc_pAuc')